In [1]:
# import math

import cv2
import json
from math import *
import mediapipe as mp
import numpy as np

In [2]:
# 벡터간의 사잇각도 측정 - 팔 각도, 손가락 마디
# https://beanexpert.tistory.com/entry/Vision-%EB%91%90-%EC%A7%81%EC%84%A0-%EC%82%AC%EC%9D%B4%EC%9D%98-%EA%B0%81%EB%8F%84-%EC%82%AC%EC%9D%B4%EA%B0%81-%EA%B5%AC%ED%95%98%EA%B8%B0
#
# def toVec(head, tail):
#     head = [head.x, head.y]
#     tail = [tail.x, tail.y]
#     head = np.array(head); tail = np.array(tail)
#     return acos(np.dot(head, tail)/(np.linalg.norm(head)*np.linalg.norm(tail)))

def toVec(head, tail):
    return [ tail.x - head.x, tail.y - head.y, tail.z - head.z ]

def vec2Rotate(head, tail):
    vecIn = head[0]*tail[0] + head[1]*tail[1] + head[2]*tail[2]
    if vecIn == 0:
        return 0
    vec01 = sqrt(pow(head[0], 2) + pow(head[1], 2) + pow(head[2], 2))
    vec02 = sqrt(pow(tail[0], 2) + pow(tail[1], 2) + pow(tail[2], 2))
    return acos(vecIn/(vec01*vec02))


def fingerRotation(hand):
    fingers = [
        [[0, 1], [1, 2], [2, 3], [3, 4]],        # 엄지 손가락
        [[0, 5], [5, 6], [6, 7], [7, 8]],        # 중지 손가락
        [[0, 9], [9, 10], [10, 11], [11, 12]],   # 검지 손가락
        [[0, 13], [13, 14], [14, 15], [15, 16]], # 약지 손가락
        [[0, 17], [17, 18], [18, 19], [19, 20]]  # 소지 손가락
    ]
    fingerLines = []
    joints = []
    for finger in fingers:
        temp = []
        for fl in finger:
            temp.append(toVec(hand.landmark[fl[0]], hand.landmark[fl[1]]))
        fingerLines.append(temp)
    for fl in fingerLines:
        for joint in range(3):
            joints.append(vec2Rotate(fl[joint], fl[joint + 1]))
    return joints

In [3]:



# cv2 이미지 프로세스 처리 - 포즈
def simpleCV2Process_pose(image):
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# cv2 이미지 프로세스 처리 - 손
def simpleCV2Process_hands(image):
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results


def lowerArmX(up, mid, low):
    upperArm = toVec(up, mid)
    lowerArm = toVec(mid, low)
    # print(upperArm, lowerArm)
    # arms = [
    #     [[11, 13], [13, 15]],        # 왼팔 각도
    #     [[12, 14], [14, 16]]         # 오른팔 각도
    # ]
    return vec2Rotate(upperArm, lowerArm)

In [4]:
# 어깨 관절 각도 계산
def upperArmX(sho, arm):
    return abs(atan((arm.y - sho.y)/(arm.x - sho.x)))

def upperArmZ(sho, arm):
    # print(degrees(abs(tan((arm.z - sho.z)/(arm.y - sho.y)))))
    # print((arm.z - sho.z)/(arm.y - sho.y))
    # print(arm.z, sho.z, arm.y, sho.y)
    # print(arm.z - sho.z, arm.y - sho.y)
    # print()
    return abs(tan((arm.z - sho.z)/(arm.y - sho.y)))





def shoulder_ANGLE_Z(landmarks, type):
    if type == "LEFT":
        hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
        shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
        elbow = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y])
    else:
        hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y])
        shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y])
        elbow = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])
    radians = np.arctan2(elbow[1]-shoulder[1], elbow[0]-shoulder[0]) - np.arctan2(hip[1]-shoulder[1], hip[0]-shoulder[0])
    angle = np.abs(degrees(radians))
    if angle > 180.0:
        angle = 360 - angle
    return np.radians(angle)



def shoulder_ANGLE(landmarks, type):
    if type == "LEFT":
        Ashoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y])
        shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
        elbow = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y])
    else:
        Ashoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
        shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y])
        elbow = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])

    radians = np.arctan2(elbow[1]-shoulder[1], elbow[0]-shoulder[0]) - np.arctan2(Ashoulder[1]-shoulder[1], Ashoulder[0]-shoulder[0])
    angle = np.abs(degrees(radians))
    if angle > 180.0:
        angle = 360 - angle
    angle -= 90
    # print(angle)
    return np.radians(90 - angle)


def calculate_angle(landmarks, type):
    if type == "LEFT":
        shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
        elbow = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y])
        wrist = np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
    else:
        shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y])
        elbow = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])
        wrist = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])

    radians = np.arctan2(wrist[1]-elbow[1], wrist[0]-elbow[0]) - np.arctan2(shoulder[1]-elbow[1], shoulder[0]-elbow[0])
    # angle = np.abs(radians*180.0/np.pi)
    angle = np.abs(degrees(radians))

    if angle > 180.0:
        angle = 360 - angle
    #     가중치 140 ( 최대 140 )
    return np.radians( (180-angle)/180*140 )

In [5]:
# 사용법
# degrees(toVec(pose[10], pose[12]))

In [22]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

mp_hands = mp.solutions.hands

# cap = cv2.VideoCapture("MOV000249438_700X466.webm")
# cap = cv2.VideoCapture("MOV000253928_sign_700X466.webm")
# cap = cv2.VideoCapture(0)

frames = {}
fps = 0
# correction = []
import time

cap = cv2.VideoCapture("yee.webm")
# cap = cv2.VideoCapture(0)
# # pose 감지
# time.sleep(3)
leftShoulder = 0
rightShoulder = 0
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success: break
        image, results = simpleCV2Process_pose(image)
        # mp_drawing.draw_landmarks( image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        # 좌우 반전되어 있었음
        # cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        # cv2.imshow('MediaPipe Pose', image)
        # if cv2.waitKey(10) & 0xFF == ord('q'):
        #     break

        frame = {}



        if results.pose_landmarks:
            if not leftShoulder:
                leftShoulder = (results.pose_landmarks.landmark[11])
            if not rightShoulder:
                rightShoulder = (results.pose_landmarks.landmark[12])

            landmarks = results.pose_landmarks.landmark
            # print(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value])

            # print(calculate_angle(landmarks, "LEFT"))

            frame["left"] = {"upperArm" : [shoulder_ANGLE(landmarks, "LEFT"),
                                           shoulder_ANGLE_Z(landmarks, "LEFT")],
                                           # upperArmZ(leftShoulder, results.pose_landmarks.landmark[13])],
                             "lowerArm" : [calculate_angle(landmarks, "LEFT")]}


                             # "lowerArm" : upperArmZ(results.pose_landmarks.landmark[13], results.pose_landmarks.landmark[15])}


            frame["right"] = {"upperArm" : [shoulder_ANGLE(landmarks, "RIGHT"),
                                           shoulder_ANGLE_Z(landmarks, "RIGHT")],
                                           # upperArmZ(leftShoulder, results.pose_landmarks.landmark[14])],
                             "lowerArm" : [calculate_angle(landmarks, "RIGHT")]}

            frames[f"frame_{fps}"] = frame
            fps += 1


            # if len(correction) <= 4:
            #     correction.append(upperArmZ(leftShoulder, results.pose_landmarks.landmark[13]))
            # else:
            #     m = (correction[4] - correction[0]) / 4
            #     b = correction[0] - m*1
            #     # print(f"y = {m}x + {b}")
            #     for relateveFps in range(1, 6):
            #         frames[f"frame_{fps-(6-relateveFps)}"]["left"]["upperArm"][1] = m*relateveFps + b
            #         # print(m*relateveFps + b)
            #
            #
            #     correction = []
            #     correction.append(upperArmZ(leftShoulder, results.pose_landmarks.landmark[13]))

        # print(lowerArmX(results.pose_landmarks.landmark[12], results.pose_landmarks.landmark[15], results.pose_landmarks.landmark[16]))


# 손가락
cap = cv2.VideoCapture("yee.webm")
fps = 0
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        fps += 1
        success, image = cap.read()
        if not success: break

        image, results = simpleCV2Process_hands(image)
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                # mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                #                           mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                #                           mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                #                           )
                hand_facing = results.multi_handedness[num].classification[0].label

                hands_data = fingerRotation(hand)
                frame = {
                    "Thumb" : {
                        "1" : hands_data[0],
                        "2" : hands_data[1],
                        "3" : hands_data[2],
                    },
                    "Index" : {
                        "1" : hands_data[3],
                        "2" : hands_data[4],
                        "3" : hands_data[5],
                    },
                    "Middle" : {
                        "1" : hands_data[6],
                        "2" : hands_data[7],
                        "3" : hands_data[8],
                    },
                    "Ring" : {
                        "1" : hands_data[9],
                        "2" : hands_data[10],
                        "3" : hands_data[11],
                    },
                    "Pinky" : {
                        "1" : hands_data[12],
                        "2" : hands_data[13],
                        "3" : hands_data[14],
                    }
                }
                if hand_facing.lower() == "left":
                    frames[f"frame_{fps}"]["right"]["hand"] = frame
                else:
                    frames[f"frame_{fps}"]["left"]["hand"] = frame


cap.release()
cv2.destroyAllWindows()
print("END")

END


In [23]:
#                                       _    _
#                                      | |  (_)
#   ___   ___   _ __  _ __   ___   ___ | |_  _   ___   _ __
#  / __| / _ \ | '__|| '__| / _ \ / __|| __|| | / _ \ | '_ \
# | (__ | (_) || |   | |   |  __/| (__ | |_ | || (_) || | | |
#  \___| \___/ |_|   |_|    \___| \___| \__||_| \___/ |_| |_|
#


last_frame = int([*frames.keys()][-1].replace("frame_", ""))
__lastStep__ = last_frame // 5 + (0 if last_frame%5 == 0 else 1)
for __frame__ in range(__lastStep__):
    if __frame__ * 5 <= last_frame:
        if "hand" in frames[f"frame_{__frame__ * 5}"]["left"]:
            for __finger__ in ["Thumb", "Index", "Middle", "Ring", "Pinky"]:
                # 1차 함수 생성
                for __joint__ in ["1", "2", "3"]:
                    fr = frames[f"frame_{__frame__ * 5}"]["left"]["hand"][__finger__][__joint__]
                    # 만약 마지막 프레임에 손이 없으면...
                    for __holy__ in range(4, -1, -1):
                        if "hand" in frames[f"frame_{__frame__ * 5 + __holy__}"]["left"]:
                            ls = frames[f"frame_{__frame__ * 5 + __holy__}"]["left"]["hand"][__finger__][__joint__]
                            break
                    m = (ls - fr) / 4
                    b = fr
                    for __relateve__ in range(5):
                        __rFrame__ = __frame__ * 5 + __relateve__
                        if "hand" in frames[f"frame_{__rFrame__}"]["left"]:
                            frames[f"frame_{__rFrame__ }"]["left"]["hand"][__finger__][__joint__] = m * __relateve__ + b

with open('test.json', 'w', encoding='utf-8') as make_file:
    json.dump(frames, make_file, indent="\t")
print("Successfully")

Successfully
